# Final Project

## Imports

In [1]:
# <include-final_project/utils.py>

In [2]:
# <imports>
import numpy as np
import pandas as pd
import plotly.io as pio


from final_project import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "seaborn"
pio.renderers.default = "vscode+notebook_connected"

## Summary

This is the exploration of a spread trading stategy involving BTCUSDT and BTCUSDT perpetual futures contracts on the [binance exchange](https://www.binance.us/en/home).

### Overview
The basic strategy is to track the spread between the spot rate and perpetual futures rate and short it when it is high and buy it when it sufficiently negative. The strategy is effected by having an absolute value band where positions are opened if the value of the spread is greater than the upper end of the band and closed when the value of the spread drops back below the lower bound of the band. If the value of the spread is positive, short positions are opened and closed and if the value of the spread is negative, long positions are opened and closed. We consider the spread to be the return on the perptual future contract over return on the spot price. Thus, if the spread goes above the upper band of the positive band, the short position that will be established will consist of shorting the perpetual futures contract and going long the underlying asset.

### Funding Rate
The perptual futures contract has a funding rate that are periodic payments made to either short or long traders based on the difference in the perpetual futures price and the spot price. When the market is bullish - perpetual futures price greater than the spot price - the funding rate is positive and long traders pay short traders. When it is bearish, the funding rate is negative and short traders pay long traders.

Funding rate payments are made every 8 hours starting at 00:00 UTC and only gets paid if positions are held at the designated time.

The actual rate has two components, an interest rate and a premium. The interest rate is set by the exchange and may change based on market conditions, such as changes in the federal funds rate. The current interest rate is 0.01% per eight hours, which equates to 0.03% per day or 10.8% per year. The premium is determined based on the bid ask spread relative to an index formed from a bucket of prices from major spot market exchanges [need to understand better](https://www.binance.com/en/support/faq/360033525031). At this point, to begin evaluating the strategy, we use the historical funding rates as provided by the exhange, and have on the todo list a fuller understanding of the mechanics of determining the funding rate.

### Liquidation
Both assets are subject to automatic liquidation when collateral = initial collateral + realized and unrealized profits and losses is less than the maintenance margin. Maintenance margin is determined based on position size an leverage. Perpetual futures contracts can be traded with leverage up to 125x. [need to understand better](https://www.binance.com/en/support/faq/360033525271)

#### Questions
* How much volume has been traded in each of the securities over the last several years?
* What is the variance and kurtosis of the spread? Does the funding rate mute the volatility of the underlying asset such that it is a high quality spread?


### Markets

In [3]:
df_exch = utils.get_exchange_info()
df_exch.loc["BTCUSDT"]

status                                                                  TRADING
baseAsset                                                                   BTC
baseAssetPrecision                                                            8
quoteAsset                                                                 USDT
quotePrecision                                                                8
quoteAssetPrecision                                                           8
baseCommissionPrecision                                                       8
quoteCommissionPrecision                                                      8
orderTypes                    [LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...
icebergAllowed                                                             True
ocoAllowed                                                                 True
quoteOrderQtyMarketAllowed                                                 True
isSpotTradingAllowed                    

## Perpetual Contract

In [56]:
df_perpetual = utils.get_continuous_contracts(pair="BTCUSDT", start_time="2020-05-01", interval="8h")

In [57]:
fig = utils.make_price_volume_chart(
    df_perpetual,
    title="BTCUSDT Perpetual Contracts 8 Hour Intervals"
)
fig.show()

In [51]:
fig = utils.make_overview_chart(df_perpetual.per_return, title="BTCUSDT Perpetual", subtitle_base="Log Returns")
fig.show()

## Spot Prices

These are the same prices as above.

In [52]:
df_spot = utils.get_klines(symbol="BTCUSDT", start_time="2020-05-01", interval="8h")
fig = utils.make_price_volume_chart(df_spot, title="BTCUSDT Spot Price OHLC 8 Hour Intervals")
fig.show()

In [54]:
fig = utils.make_overview_chart(
    df_spot.per_return, title="BTCUSDT Spot",
    subtitle_base="Log Returns"
)
fig.show()

## Funding Rate

It looks like the funding rate rarely goes negative. Is that because the perpetual price rarely goes below the spot price or is there something structural that may present and arbitrage opportunity going on?

In [55]:
df_funding = utils.get_funding_rate_history(symbol="BTCUSDT", start_time="2020-05-01")
fig = df_funding.fundingRate.plot(title="BTCUSDT Funding Rate")
fig.update_traces(line=dict(width=1))
fig.update(layout_showlegend=False)
fig.show()

## Spread

This is of the spread itself - the percentage difference between the perpetual and spot prices and excludes a spread outlier on 2020-12-21 of 0.018.

* Should we also look st the spread between the funding rate and the spread between the perpetual contract and underlying asset.

In [61]:
spread = df_perpetual.close / df_spot.close - 1
spread.name = "spread"
fig = utils.make_2_yaxis_lines(
    spread[spread.abs() < .015], df_funding.fundingRate,
    title="Perpetual - Spot Spread vs. Funding Rate"
)
print(f"correlation: {spread[spread.abs() < .015].corr(df_funding.fundingRate):0.4f}")
fig.show()

correlation: 0.6357


In [60]:
fig = (spread[spread.abs() < .015] - df_funding.fundingRate).plot()
fig.update_traces(line=dict(width=1))
fig.update(
    layout_showlegend=False,
    layout_title="Perpetual-Spot Spread Over Funding Rate"
)
fig.show()

Here we look at the spread between the returns on the perpetual contract and underlying asset, again excluding the outlier on 2020-12-21. This could be a different strategy - just playing the reversion to the mean of the returns spread without regard to the funding rate cost.

In [64]:
spread = df_perpetual.per_return - df_spot.per_return
spread.name = "spread"
fig = utils.make_2_yaxis_lines(
    spread[spread.abs() < .015], df_funding.fundingRate,
    title="Perpetual - Spot Return Spread vs. Funding Rate"
)
print(f"correlation: {spread[spread.abs() < .015].corr(df_funding.fundingRate):0.4f}")
fig.show()

correlation: -0.1496


In [67]:
net_spread = spread[spread.abs() < .015] - df_funding.fundingRate
fig = net_spread.plot()
fig.update_traces(line=dict(width=1))
fig.update(
    layout_showlegend=False,
    layout_title="Perpetual-Spot Spread Over Funding Rate"
)
fig.show()

Here we look at the return on the perpetual less the funding rate (which the long traders of perpetual contracts pay to the short traders of perpetual contracts) versus the return on the underlying asset.

In [69]:
fig = utils.make_overview_chart(
    net_spread,
    title="Net Perpetual Return Over Spot Return",
    subtitle_base="Spread"
)
fig.show()